In [2]:
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns
sns.set_style('white')

import sys
#sys.path.append("C:/Users/user/Desktop/ipynb/misshapen/")
sys.path.append("C:/Users/Yimeng/Documents/GitHub/")
from misshapen import shape, nonshape

import util
import imp
imp.reload(util)
from __future__ import print_function

import os
import glob

# Load behavioral dataframe

In [3]:
dfmove = pd.read_csv('behavior_all.csv',index_col=0)
dfmove

,cue_end,cue_start,delay_end,delay_start,duration,finger,move_end,move_start,subject
0,9079,7079,680.0,1160.0,1520.0,5.0,9759.0,8239.0,bp
1,13079,11079,640.0,840.0,1800.0,5.0,13719.0,11919.0,bp
2,17079,15079,400.0,720.0,1680.0,5.0,17479.0,15799.0,bp
3,21079,19079,600.0,800.0,1800.0,1.0,21679.0,19879.0,bp
4,25079,23079,560.0,880.0,1680.0,5.0,25639.0,23959.0,bp
5,29079,27079,240.0,1160.0,1080.0,2.0,29319.0,28239.0,bp
6,33079,31079,840.0,1440.0,1400.0,3.0,33919.0,32519.0,bp
7,37079,35079,80.0,640.0,1440.0,4.0,37159.0,35719.0,bp
8,41079,39079,760.0,1000.0,1760.0,1.0,41839.0,40079.0,bp
9,45079,43079,760.0,1040.0,1720.0,1.0,45839.0,44119.0,bp


# Load neural data

In [4]:
Fs = 1000 # sampling rate
f_range = (10,30) # frequency range of oscillation of interest

dfs=[]
data_paths, all_regions = util.load_paths()


for key in data_paths:
    df_subj = dfmove[dfmove['subject']==key]
    for i in data_paths[key]['1']:
        electrode=i.split('\\')[1]
        electrode = electrode[:-4]
        x = np.load(i) # voltage series
        
        findpt_kwargs = {'filter_fn':nonshape.bandpass_default,
                'filter_kwargs': {'w':3}}
        define_true_oscillating_periods_kwargs = {'ampdiff_th':.5, 'timediff_th':.6}
        df_P,df_T = shape.compute_shape_by_cycle(x, f_range, Fs,
                                          findpt_kwargs=findpt_kwargs,
                                          define_true_oscillating_periods_kwargs=define_true_oscillating_periods_kwargs)
       
        df_T_part = df_T[['amp_mean', 'period', 'ptsym_time', 'rdsym_time',
                   'sample', 'sample_lastE', 'sample_nextE', 'sample_zeroxD', 'sample_zeroxR',
                   'trough_sharp', 'trough_time', 'trough_volt', 'whole_decay_time',
                   'whole_rise_time', 'whole_rise_volt', 'oscillating_amp', 'oscillating_amp_time']]
        df_T_part['subject']=key
        #df_T_filtered=util.cutoff(0,df_T)
        df_no_bad_trials = util.drop_bad_trials(df_subj,1500,'delay_start',True,True)
        df_no_bad_trials = util.drop_bad_trials(df_subj,2000,'delay_end',True,True)
        rdsym,average,sem=util.plot_shape(x,df_T_part,df_no_bad_trials,-1000,1000,Fs,key,electrode)
        dfs.append(df_T_part)

C:\Users\Yimeng\Anaconda3\lib\site-packages\numpy\core\_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)
C:\Users\Yimeng\Anaconda3\lib\site-packages\numpy\core\_methods.py:70: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


C:/Users/Yimeng/Documents/GitHub\misshapen\shape.py:738: RuntimeWarning: invalid value encountered in true_divide
  trough_stats['half_decay_time']
C:/Users/Yimeng/Documents/GitHub\misshapen\shape.py:740: RuntimeWarning: invalid value encountered in true_divide
  trough_stats['half_rise_time']
C:/Users/Yimeng/Documents/GitHub\misshapen\shape.py:782: RuntimeWarning: invalid value encountered in true_divide
  peak_stats['half_decay_time']
C:/Users/Yimeng/Documents/GitHub\misshapen\shape.py:784: RuntimeWarning: invalid value encountered in true_divide
  peak_stats['half_rise_time']
C:\Users\Yimeng\Anaconda3\lib\site-packages\ipykernel\__main__.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Yimeng\Anaconda3\lib\site-packages\numpy\core\_methods.py:59: Ru

C:/Users/Yimeng/Documents/GitHub\misshapen\shape.py:738: RuntimeWarning: invalid value encountered in true_divide
  trough_stats['half_decay_time']
C:/Users/Yimeng/Documents/GitHub\misshapen\shape.py:740: RuntimeWarning: invalid value encountered in true_divide
  trough_stats['half_rise_time']
C:/Users/Yimeng/Documents/GitHub\misshapen\shape.py:782: RuntimeWarning: invalid value encountered in true_divide
  peak_stats['half_decay_time']
C:/Users/Yimeng/Documents/GitHub\misshapen\shape.py:784: RuntimeWarning: invalid value encountered in true_divide
  peak_stats['half_rise_time']
C:\Users\Yimeng\Anaconda3\lib\site-packages\ipykernel\__main__.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Yimeng\Anaconda3\lib\site-packages\numpy\core\_methods.py:59: Ru

C:/Users/Yimeng/Documents/GitHub\misshapen\shape.py:740: RuntimeWarning: invalid value encountered in true_divide
  trough_stats['half_rise_time']
C:/Users/Yimeng/Documents/GitHub\misshapen\shape.py:782: RuntimeWarning: invalid value encountered in true_divide
  peak_stats['half_decay_time']
C:\Users\Yimeng\Anaconda3\lib\site-packages\ipykernel\__main__.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Yimeng\Anaconda3\lib\site-packages\numpy\core\_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)
C:\Users\Yimeng\Anaconda3\lib\site-packages\numpy\core\_methods.py:70: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


C:/Users/Yimeng/Documents/GitHub\misshapen\shape.py:738: RuntimeWarning: invalid value encountered in true_divide
  trough_stats['half_decay_time']
C:/Users/Yimeng/Documents/GitHub\misshapen\shape.py:740: RuntimeWarning: invalid value encountered in true_divide
  trough_stats['half_rise_time']
C:/Users/Yimeng/Documents/GitHub\misshapen\shape.py:782: RuntimeWarning: invalid value encountered in true_divide
  peak_stats['half_decay_time']
C:/Users/Yimeng/Documents/GitHub\misshapen\shape.py:784: RuntimeWarning: invalid value encountered in true_divide
  peak_stats['half_rise_time']
C:\Users\Yimeng\Anaconda3\lib\site-packages\ipykernel\__main__.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Yimeng\Anaconda3\lib\site-packages\numpy\core\_methods.py:59: Ru

...

C:/Users/Yimeng/Documents/GitHub\misshapen\shape.py:738: RuntimeWarning: invalid value encountered in true_divide
  trough_stats['half_decay_time']
C:/Users/Yimeng/Documents/GitHub\misshapen\shape.py:740: RuntimeWarning: invalid value encountered in true_divide
  trough_stats['half_rise_time']
C:/Users/Yimeng/Documents/GitHub\misshapen\shape.py:782: RuntimeWarning: invalid value encountered in true_divide
  peak_stats['half_decay_time']
C:/Users/Yimeng/Documents/GitHub\misshapen\shape.py:784: RuntimeWarning: invalid value encountered in true_divide
  peak_stats['half_rise_time']
C:\Users\Yimeng\Anaconda3\lib\site-packages\ipykernel\__main__.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Yimeng\Anaconda3\lib\site-packages\numpy\core\_methods.py:59: Ru

C:/Users/Yimeng/Documents/GitHub\misshapen\shape.py:738: RuntimeWarning: invalid value encountered in true_divide
  trough_stats['half_decay_time']
C:/Users/Yimeng/Documents/GitHub\misshapen\shape.py:740: RuntimeWarning: invalid value encountered in true_divide
  trough_stats['half_rise_time']
C:/Users/Yimeng/Documents/GitHub\misshapen\shape.py:782: RuntimeWarning: invalid value encountered in true_divide
  peak_stats['half_decay_time']
C:/Users/Yimeng/Documents/GitHub\misshapen\shape.py:784: RuntimeWarning: invalid value encountered in true_divide
  peak_stats['half_rise_time']
C:\Users\Yimeng\Anaconda3\lib\site-packages\ipykernel\__main__.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Yimeng\Anaconda3\lib\site-packages\numpy\core\_methods.py:59: Ru

C:/Users/Yimeng/Documents/GitHub\misshapen\shape.py:738: RuntimeWarning: invalid value encountered in true_divide
  trough_stats['half_decay_time']
C:/Users/Yimeng/Documents/GitHub\misshapen\shape.py:740: RuntimeWarning: invalid value encountered in true_divide
  trough_stats['half_rise_time']
C:/Users/Yimeng/Documents/GitHub\misshapen\shape.py:782: RuntimeWarning: invalid value encountered in true_divide
  peak_stats['half_decay_time']
C:/Users/Yimeng/Documents/GitHub\misshapen\shape.py:784: RuntimeWarning: invalid value encountered in true_divide
  peak_stats['half_rise_time']
C:\Users\Yimeng\Anaconda3\lib\site-packages\ipykernel\__main__.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Yimeng\Anaconda3\lib\site-packages\numpy\core\_methods.py:59: Ru

C:/Users/Yimeng/Documents/GitHub\misshapen\shape.py:738: RuntimeWarning: invalid value encountered in true_divide
  trough_stats['half_decay_time']
C:/Users/Yimeng/Documents/GitHub\misshapen\shape.py:740: RuntimeWarning: invalid value encountered in true_divide
  trough_stats['half_rise_time']
C:/Users/Yimeng/Documents/GitHub\misshapen\shape.py:782: RuntimeWarning: invalid value encountered in true_divide
  peak_stats['half_decay_time']
C:/Users/Yimeng/Documents/GitHub\misshapen\shape.py:784: RuntimeWarning: invalid value encountered in true_divide
  peak_stats['half_rise_time']
C:\Users\Yimeng\Anaconda3\lib\site-packages\ipykernel\__main__.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Yimeng\Anaconda3\lib\site-packages\numpy\core\_methods.py:59: Ru

C:/Users/Yimeng/Documents/GitHub\misshapen\shape.py:738: RuntimeWarning: invalid value encountered in true_divide
  trough_stats['half_decay_time']
C:/Users/Yimeng/Documents/GitHub\misshapen\shape.py:740: RuntimeWarning: invalid value encountered in true_divide
  trough_stats['half_rise_time']
C:/Users/Yimeng/Documents/GitHub\misshapen\shape.py:782: RuntimeWarning: invalid value encountered in true_divide
  peak_stats['half_decay_time']
C:/Users/Yimeng/Documents/GitHub\misshapen\shape.py:784: RuntimeWarning: invalid value encountered in true_divide
  peak_stats['half_rise_time']
C:\Users\Yimeng\Anaconda3\lib\site-packages\ipykernel\__main__.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [5]:
subj_keys=list(data_paths.keys())





In [54]:
dfn=pd.concat(dfs) 
dfn.to_csv('./neural_all.csv')

dfneural = pd.read_csv('neural_all.csv',index_col=0) 

In [52]:
def calculate_average(trial,label):
    total = 0
    if(len(trial)!=0):
        for num in trial[label]:
            total += num;
        return total/len(trial)
    return 0

In [62]:
dfneural_avg = dfneural
dfneural_nomove=pd.DataFrame()
dfneural_nomove

""


In [65]:
interval = 500
for subj in data_paths:
    df_subj_move = dfmove[dfmove['subject']==subj]
    df_subj_neural=dfneural[dfneural['subject']==subj]
    start=df_subj_move['move_start']
    end = df_subj_move['move_end']

    for index in range(len(start)):
        try:
            if index != len(start)-1:
                nomove= df_subj_neural[(df_subj_neural['sample']>=end[index]+interval) & (df_subj_neural['sample']<=start[index+1]+interval)]
            
                avg_nomove=calculate_average(trial,'rdsym_time')
                dfneural_nomove=pd.concat([dfneural_nomove,nomove],ignore_index=True)
        
                for sample in nomove['sample']:
                    dfneural_nomove.loc[dfneural_nomove['sample']==sample,'average rdsym']=avg_nomove
        except KeyError:
            avg=0   
        try:
            trial = df_subj_neural[(df_subj_neural['sample']<=end[index]) & (df_subj_neural['sample']>=start[index])]
        
            total = 0
            avg = 0
            if(len(trial)!=0):
                avg = calculate_average(trial,'rdsym_time')
            
                for sample in trial['sample']:
                    dfneural_avg.loc[dfneural_avg['sample']==sample,'average rdsym']=avg
        except KeyError:
            avg=0
        

In [68]:
dfneural_avg
dfneural_avg.to_csv('./dfneural_avg.csv')

In [67]:
dfneural_nomove
dfneural_nomove.to_csv('./dfneural_nomove.csv')